In [ ]:
# Configuration
learning_rate = 0.01  # Learning rate
epochs = 30  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 32  # Batch size

# Train/valid/test split
# https://github.com/danielegrattarola/spektral/blob/master/examples/graph_prediction/custom_dataset.py
idxs = np.random.permutation(len(d))
split_va, split_te = int(0.8 * len(d)), int(0.9 * len(d))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
data_tr = d[idx_tr]
data_va = d[idx_va]
data_te = d[idx_te]

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net(Model):
    def __init__(self):
        super().__init__()
        self.eccconv1 = ECCConv(8, activation='relu')
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a, e = inputs
        print(x.shape)
        x = self.eccconv1([x, a, e])
        print(x.shape)
        x =self.flatten(x)
        print(x.shape)
        x = self.dense(x)
        print(x.shape)
        
        return x

model = Net()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])